In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextStreamer, GenerationConfig

model_name='davidkim205/komt-mistral-7b-v1'
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer)

def gen(x):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=1024,
        early_stopping=True,
        do_sample=True,
    )
    q = f"[INST]{x} [/INST]"
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"\n\n### Response: "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [7]:
import platform
import os
import pandas as pd


TEST_NAME = "llm1"

# 데이터 관련
os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'
DATA_PATH = PRE_PATH + "data/" # 대회에서 제공한 데이터
OUTPUT_PATH = PRE_PATH + "output/" # 모델의 출력 값
CHECKPOINT_PATH = PRE_PATH + "checkpoint/" # 모델의 최종 출력 값
PREDICTION_PATH = PRE_PATH + "prediction/" # 최종 예측 값
LOG_PATH = PRE_PATH + "log/"
TRAIN_PATH = DATA_PATH + "train_new2_temp.csv"
VALID_PATH = DATA_PATH + "dev_new2.csv"
TEST_PATH = DATA_PATH + "test.csv"
SUBMIT_PATH = PREDICTION_PATH + "submission_" + TEST_NAME + ".csv"

train_df = pd.read_csv(os.path.join(TRAIN_PATH))
val_df = pd.read_csv(os.path.join(VALID_PATH))

sampled_df = train_df.sample(n=10, random_state=42)  # random_state는 결과를 재현 가능하게 합니다.
combined_string = '\n'.join(sampled_df.apply(lambda row: f"{row['fname']}, {row['dialogue']}, {row['summary']}, {row['topic']}", axis=1).tolist())
print(combined_string)

train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 2005년까지 문제가 해결될 것이라고 생각해요., #Person1#의 컴퓨터에 끔찍한 바이러스가 걸렸고, #Person2#는 IT 부서에게 바이러스 제거 프로그램을 실행하도록 제안했다., 컴퓨터 바이러스
train_10667, #Person1#: 우리 회사에 관심이 있는 이유는 무엇인가요?
#Person2#: 귀사는 세계적으로 유명합니다. 그래서 개인적인 발전을 위한 더 나은 기회가 있을 것이라고 생각합니다.
#Person1#: 맞습니다. 이 직무에 필요한 자격을 갖추고 있다고 생각하나요?
#Person2#: 그럼요. 제 자격은 귀사의 직무 설명에 부합합니다.
#Person1#: 하지만 이 직무에 대한 경험이 없습니다.
#Person2#: 그건 사실입니다. 하지만 배우는 것에 열정이 있고, 빠르게 배울 수 있습니다.
#Person1#: 추가 교육이 필요하다면, 교육비를 아시나요?
#Person2#: 아니요. 월급은 얼마인가요?
#Person1#: 월 800위안입니다.
#Person2#: 내일 답변을 드려도 괜찮을까요?
#Person1#: 예. 물론입니다., #Person2#는 세계적으로 유명한 #Person1#의 회사에 관심이 있으며, 경험이 없음에도 불구하고 #Person2#는 자신이 직무에 필요한 자격을 갖추고 있다고 생각합니다., 직무 인터뷰
train_5261, #Person1#: 데이비드, 우리가 여기 앉아

In [8]:
print(gen(f"{combined_string} \n\n\n 위와 같은 대화문과 요약문이 존재해. 어떤 방식으로 요약되었는지 패턴을 찾고, 나에게 설명해줄 수 있어?"))

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1658 > 1024). Running this sequence through the model will result in indexing errors


<s> [INST]train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 2005년까지 문제가 해결될 것이라고 생각해요., #Person1#의 컴퓨터에 끔찍한 바이러스가 걸렸고, #Person2#는 IT 부서에게 바이러스 제거 프로그램을 실행하도록 제안했다., 컴퓨터 바이러스
train_10667, #Person1#: 우리 회사에 관심이 있는 이유는 무엇인가요?
#Person2#: 귀사는 세계적으로 유명합니다. 그래서 개인적인 발전을 위한 더 나은 기회가 있을 것이라고 생각합니다.
#Person1#: 맞습니다. 이 직무에 필요한 자격을 갖추고 있다고 생각하나요?
#Person2#: 그럼요. 제 자격은 귀사의 직무 설명에 부합합니다.
#Person1#: 하지만 이 직무에 대한 경험이 없습니다.
#Person2#: 그건 사실입니다. 하지만 배우는 것에 열정이 있고, 빠르게 배울 수 있습니다.
#Person1#: 추가 교육이 필요하다면, 교육비를 아시나요?
#Person2#: 아니요. 월급은 얼마인가요?
#Person1#: 월 800위안입니다.
#Person2#: 내일 답변을 드려도 괜찮을까요?
#Person1#: 예. 물론입니다., #Person2#는 세계적으로 유명한 #Person1#의 회사에 관심이 있으며, 경험이 없음에도 불구하고 #Person2#는 자신이 직무에 필요한 자격을 갖추고 있다고 생각합니다., 직무 인터뷰
train_5261, #Person1#: 데이비드,

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


[/INST]이 벽지를 원하지 않았어요. 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요.
#Person1#의 

KeyboardInterrupt: 